In [14]:
from dataclasses import dataclass
from typing import List, Dict, Tuple
import heapq

In [15]:
@dataclass(order=True)
class Process:
    pid: str
    arrival: int
    cpu_burst: int

In [16]:
@dataclass
class PeriodicProcess:
    pid: str
    arrival: int
    cpu_burst: int
    period: int
    deadline: int = None

    def __post_init__(self):
        if self.deadline is None:
            self.deadline = self.period

In [17]:
def srt(processes):
    # Copy and sort process by (arrival, pid)
    not_arrived_processes = sorted(processes, key=lambda p: (p.arrival, p.pid))

    # processes currently eligible to run with remaining time
    ready_processes = []

    # The resulting process schedule
    scheduled_processes = []

    # completion time of processes
    completion = {}

    # Track remaining time for each process
    remaining_time = {p.pid: p.cpu_burst for p in processes}

    # current time
    t = 0

    # Currently running process
    current_process = None
    current_start = 0

    # Loop until there are no processes left to run
    while not_arrived_processes or ready_processes or current_process:
        # Find next event time (arrival or completion)
        next_arrival = not_arrived_processes[0].arrival if not_arrived_processes else float('inf')
        next_completion = t + remaining_time[current_process.pid] if current_process else float('inf')
        next_event = min(next_arrival, next_completion)

        # If nothing is happening, jump to next arrival
        if next_event == float('inf'):
            break

        # Save current process if it was running
        if current_process and t < next_event:
            scheduled_processes.append((current_process.pid, current_start, next_event))
            remaining_time[current_process.pid] -= (next_event - t)

            # Check if process completed
            if remaining_time[current_process.pid] == 0:
                completion[current_process.pid] = next_event
                current_process = None
            else:
                # Put back in ready queue for preemption check
                ready_processes.append(current_process)
                current_process = None

        # Advance time
        t = next_event

        # Admit all processes whose arrival time has passed (arrival <= t)
        while not_arrived_processes and not_arrived_processes[0].arrival <= t:
            # Move the process into the ready list
            ready_processes.append(not_arrived_processes.pop(0))

        # Choose the next process if not currently running
        if not current_process and ready_processes:
            # Sort by remaining time, then arrival, then pid
            ready_processes.sort(key=lambda p: (remaining_time[p.pid], p.arrival, p.pid))
            # Take the process with the shortest remaining time
            current_process = ready_processes.pop(0)
            current_start = t

    return scheduled_processes, completion

In [18]:
def tat(processes, completion):
    """Calculate Turn Around Time for each process"""
    return {p.pid: completion[p.pid] - p.arrival for p in processes}


def att(tats):
    """Calculate Average Turn Around Time"""
    return (sum(tats.values()) / len(tats)) if tats else 0.0




def print_schedule(schedule, title="Schedule"):
    """Pretty print the schedule"""
    print(f"\n{title}:")
    print("-" * 40)
    for pid, start, end in schedule:
        print(f"  {pid}: [{start:3} - {end:3}]")



def print_metrics(processes, completion, title="Metrics"):
    """Print TAT and ATT metrics"""
    tats = tat(processes, completion)
    average_tat = att(tats)
    print(f"\n{title}:")
    print(f"  TAT: {tats}")
    print(f"  ATT: {average_tat:.2f}")

In [19]:
example = [
    Process('p1', arrival=0, cpu_burst=3),
    Process('p2', arrival=2, cpu_burst=6),
    Process('p3', arrival=4, cpu_burst=4),
    Process('p4', arrival=6, cpu_burst=5),
    Process('p5', arrival=8, cpu_burst=2),
]
scheduled_processes, completion = srt(example)
tats = tat(example, completion)
att = att(tats)

In [20]:
print('TAT: ', tats)
print('ATT: ', att)

TAT:  {'p1': 3, 'p2': 13, 'p3': 4, 'p4': 14, 'p5': 2}
ATT:  7.2
